# Projeto 2 - Ciência dos Dados

Nome: Matheus Freitas Sant'Ana

Nome: Thiago Lopes David

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [114]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [115]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[MathDavid6]***

In [116]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [117]:
#Produto escolhido:
produto = 'uber'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [118]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [119]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [120]:
import os
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\matfs\Desktop\Ciência dos Dados\P2-Ciencia-dos-Dados


In [ ]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    punctuation = '[!\-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed    

In [ ]:
tweets = pd.read_excel('uber.xlsx', sheet_name = "Treinamento")
tweets_teste = pd.read_excel('uber.xlsx', sheet_name = "Teste")

In [ ]:
#Criando dataframe irrelevantes e suas frequências relativas
DataFrame_irrelevantes = tweets.loc[tweets['Classificação'] == 0]
texto_ir = ''.join(DataFrame_irrelevantes['Treinamento'])
texto_clean_ir = cleanup(texto_ir)
lista_ir = texto_clean_ir.split()
freq_ir = pd.Series(lista_ir).value_counts(True)*100
freq_ir

uber                   4.783599
o                      3.449398
um                     2.700944
de                     2.570778
e                      2.408070
eu                     2.147738
aí                     1.854865
que                    1.496909
a                      1.366743
no                     1.204035
tava                   1.204035
do                     1.171494
pegar                  0.976245
com                    0.976245
hoje                   0.943703
fui                    0.911162
na                     0.911162
pra                    0.878620
eh                     0.878620
3                      0.878620
ne                     0.878620
não                    0.878620
sou                    0.878620
senhor                 0.846079
(uns                   0.846079
cristão                0.846079
segundos               0.846079
sim                    0.846079
@brukopinho            0.846079
tocando                0.846079
                         ...   
nivel   

In [ ]:
#Criando dataframe relevantes e suas frequências relativas
DataFrame_relevantes = tweets.loc[tweets['Classificação'] == 1]
texto_r = ''.join(DataFrame_relevantes['Treinamento'])
texto_clean_r = cleanup(texto_r)
lista_r = texto_clean_r.split()
freq_r = pd.Series(lista_r).value_counts(True)*100
freq_r

uber              4.834606
de                3.265479
e                 2.756573
o                 2.629347
que               2.205259
a                 1.993215
pra               1.908397
eu                1.781170
um                1.738762
no                1.441900
q                 1.357082
do                1.145038
não               1.060221
com               1.060221
é                 0.890585
na                0.805768
da                0.763359
minha             0.763359
ele               0.763359
meu               0.720950
me                0.678541
https             0.636132
uma               0.636132
//t               0.636132
mais              0.551315
se                0.551315
vai               0.508906
tá                0.508906
em                0.466497
por               0.466497
                    ...   
partida,          0.042409
gastado           0.042409
pede              0.042409
tal               0.042409
filha             0.042409
tbmmeu            0.042409
@

In [ ]:
soma_r = len(freq_r)
soma_ir = len(freq_ir)
total = soma_r + soma_ir

#Dados Empíricos
p_relevante = soma_r/total
p_irrelevante =soma_ir/total

In [ ]:
def classificador_naive_bayes(frase):
    
    frase_clean = cleanup(frase)
    lista_frase = frase_clean.split()
    
    #P(Frase|Relevante)
    p_frase_relevante = freq_r.loc[lista_frase].prod()
    
    #P(Frase|Irrelevante)
    p_frase_irrelevante = freq_ir.loc[lista_frase].prod()
    
    #P(Relevante|Frase)
    p_relevante_frase = p_frase_relevante*p_relevante
    
    #P(Irrelevante|Frase)
    p_irrelevante_frase = p_frase_irrelevante*p_irrelevante
    
    if p_relevante_frase > p_irrelevante_frase:
        return 1 #relevante
    else:
        return 0 #irrelevante
    
print(classificador_naive_bayes("uber eh caro"))
    
    

0


C:\Users\matfs\Downloads\Anaconda\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  # Remove the CWD from sys.path while we load stuff.


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [ ]:
#Criando a coluna de verificação 
for t in range(0, len(tweets_teste['Teste'])):
    tweets_teste['Verifica'] = t

In [ ]:
i=0
for tweet in tweets_teste["Teste"]:
    
    tweet_clean = cleanup(tweet)
    lista_tweet = tweet_clean.split()
    p_relevante_frase = 0
    p_irrelevante_frase = 0
    
    for k in lista_tweet:
        
        try:
        
            #P(Frase|Relevante)
            p_frase_relevante = freq_r.loc[k].prod()

            #P(Frase|Irrelevante)
            p_frase_irrelevante = freq_ir.loc[k].prod()

            #P(Relevante|Frase)
            p_relevante_frase = p_frase_relevante*p_relevante

            #P(Irrelevante|Frase)
            p_irrelevante_frase = p_frase_irrelevante*p_irrelevante
        
        except:
            
            p_relevante_frase += 1/(total + soma_r)
            p_irrelevante_frase += 1/(total + soma_ir)
            
            
    if p_relevante_frase > p_irrelevante_frase:
        tweets_teste["Verifica"][i]=1
    
    else:
        tweets_teste["Verifica"][i]=0
        
    i+=1
    

C:\Users\matfs\Downloads\Anaconda\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\matfs\Downloads\Anaconda\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
 tweets_teste.head(10)

In [ ]:
#crosstab de quali classificação 'na coragem' em relevante ou irrrelevante e classificação de nosso naive-bayes 
Classificação=tweets_teste['Classificação']
Treinamento=tweets_teste['Verifica']
pd.crosstab(Classificação, Treinamento)


___
### Concluindo

In [ ]:
positivos_falsos=0
positivos_verdadeiros=0
negativos_falsos=0
negativos_verdadeiros=0
total=positivos_falsos+positivos_verdadeiros+negativos_falsos+negativos_verdadeiros

i=0
for t in tweets_teste['Verifica']:
    if tweets_teste['Verifica'][i]==1 and tweets_teste['Classificação'][i]==0:
        positivos_falsos+=1
    if tweets_teste['Verifica'][i]==0 and tweets_teste['Classificação'][i]==1:
        negativos_verdadeiros+=1
    if tweets_teste['Verifica'][i]==1 and tweets_teste['Classificação'][i]==1:
        positivos_verdadeiros+=1
    if tweets_teste['Verifica'][i]==0 and tweets_teste['Classificação'][i]==0:
        negativos_falsos+=1
    i+=1
    
total=positivos_falsos+positivos_verdadeiros+negativos_falsos+negativos_verdadeiros
porcentagem_positivos_falsos=(positivos_falsos/total)*100
porcentagem_negativos_verdadeiros=(negativos_verdadeiros/total)*100
porcentagem_positivos_verdadeiros=(positivos_verdadeiros/total)*100
porcentagem_negativos_falsos=(negativos_falsos/total)*100


In [ ]:
print("Porcentagem de positivos verdadeiros:{}%".format(int(porcentagem_positivos_verdadeiros)))
print("Porcentagem de positivos falsos:{}%".format(int(porcentagem_positivos_falsos)))
print("Porcentagem de negativos verdadeiros:{}%".format(int(porcentagem_negativos_verdadeiros)))
print("Porcentagem de negativos falsos:{}%".format(int(porcentagem_negativos_falsos)))

Com os resultados obtidos, obtemos uma acurácia de 51%, indicando que a performance do nosso classificador é razoável. Um exemplo de maneira de como poderíamos aprimora-lo é dando maior "peso" nasa contagens a algumas palavras específicas que já sabemos que nos dirão se a informação é relevante ou não, como por exemplo "preço" e "cancelar", palavras que apareceram na maioria dos tweets relevantes. Além disso, para melhorar o nosso classificador, poderíamos ter analisado mais a fundo as mensagens com dupla negação ou sarcasmo que certamente impactam no resultado de nosso classificador. Não podemos utilizar o próprio classificador para gerar mais amostras de treinamento, pois já que o nosso classificador já está embutido com erros, utiliz-alo para gerar mais amostras diminuiria a acurácia drasticamente. Outros cenários para utilizar nosso classificador senão o do projeto, para calcular probabilidades em tempo real de acontecimentose e enquetes, prática utilizada pela FOX sports, por exemplo, que diz ao público qual a chance de cada time ser campeão brasileiro com base nos tweets que os torcedores fazem.

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

Referencias utilizadas:

-https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

-https://www.quora.com/What-is-Laplacian-smoothing-and-why-do-we-need-it-in-a-Naive-Bayes-classifier